# Question 1: Show the top 5 items bought (count) for the time period between 9:00AM inclusive and 11:00PM exclusive

In [1]:
from pyspark import SparkContext, SparkConf
cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/04 02:46:24 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/05/04 02:46:24 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/05/04 02:46:25 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/05/04 02:46:25 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("app").config("spark.some.config.option","some-value").getOrCreate()

In [4]:
from pyspark.sql.functions import *

In [20]:
df = spark.read.csv("BreadBasket_DMS.csv", header=True, inferSchema=True)


In [21]:
df.show(5)

+----------+--------+-----------+-------------+
|      Date|    Time|Transaction|         Item|
+----------+--------+-----------+-------------+
|2016-10-30|09:58:11|          1|        Bread|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:07:57|          3|Hot chocolate|
|2016-10-30|10:07:57|          3|          Jam|
+----------+--------+-----------+-------------+
only showing top 5 rows



In [22]:
#filter the data to include only transactions between 9:00AM inclusive and 11:00PM exclusive
df = df.filter((hour(df["Time"]) >= 9) & (hour(df["Time"]) < 23))


In [23]:
#group the data
item_counts = df.groupBy("Item").count()

In [24]:
#sort results
top_items = item_counts.orderBy(desc("count")).limit(5)


In [25]:
#display the top 5 items
top_items.show()

+------+-----+
|  Item|count|
+------+-----+
|Coffee| 5259|
| Bread| 3151|
|   Tea| 1414|
|  Cake| 1017|
|Pastry|  797|
+------+-----+



# Question 2: Summarize the number of entities by “rpt_area_desc

In [29]:
df2 = spark.read.format("csv").option("header", True).option("delimiter", ";").load("Restaurants_in_Durham_County_NC.csv")


In [30]:
entity_counts = df2.groupBy("rpt_area_desc").count()
#convert the counts to a string and concatenate with the "rpt_area_desc" column
summary = entity_counts.selectExpr("concat(rpt_area_desc, ', ', cast(count as string)) as summary")
#display the summary
summary.show(truncate=False)

+-------------------------+
|summary                  |
+-------------------------+
|Bed&Breakfast Home, 4    |
|Summer Camps, 4          |
|Institutions, 30         |
|Local Confinement, 2     |
|Mobile Food, 147         |
|School Buildings, 89     |
|null                     |
|Summer Food, 242         |
|Swimming Pools, 420      |
|Day Care, 173            |
|Tattoo Establishments, 32|
|Residential Care, 154    |
|Bed&Breakfast Inn, 2     |
|Adult Day Care, 5        |
|Lodging, 62              |
|Food Service, 1093       |
+-------------------------+



# Question 4: 20 Points
Dataset: romeo-juliet-pg1777.txt
Solve: WordCount
Do a word count exercise using pyspark. Ignore punctuation, and normalize to lower case. Accept only the characters in this set: [0-9a-zA-Z]

In [5]:
#read the text file
text_file = sc.textFile("romeo-juliet-pg1777.txt")

#split lines into words and normalize to lower case
words = text_file.flatMap(lambda line: line.lower().split())

#remove punctuation and chars
import re
regex = re.compile('[^0-9a-zA-Z]')
clean_words = words.map(lambda word: regex.sub('', word))

#count occurrences of words
word_counts = clean_words.countByValue()

#print results
for word, count in sorted(word_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{word}: {count}")

and: 773
the: 748
i: 585
to: 580
a: 483
of: 474
is: 383
that: 365
my: 360
in: 328
you: 323
thou: 277
not: 275
with: 268
for: 267
me: 265
this: 258
it: 237
be: 234
but: 186
as: 168
what: 166
thy: 164
rom: 163
so: 159
her: 156
o: 155
by: 154
will: 152
nurse: 150
romeo: 139
his: 139
thee: 138
love: 137
have: 131
or: 126
he: 120
your: 117
jul: 117
she: 114
shall: 110
no: 108
all: 103
him: 100
come: 98
from: 93
do: 93
friar: 92
if: 91
are: 90
an: 88
then: 85
ill: 84
enter: 82
good: 82
on: 81
now: 81
here: 80
at: 77
go: 76
we: 74
man: 72
death: 69
well: 69
one: 68
more: 68
lady: 68
night: 68
may: 67
our: 65
which: 64
ben: 64
hath: 64
there: 64
mer: 62
juliet: 61
am: 60
such: 59
how: 59
they: 59
some: 58
too: 57
up: 56
sir: 56
would: 56
when: 55
tybalt: 55
art: 55
should: 53
out: 53
cap: 53
give: 52
where: 52
say: 52
than: 49
was: 49
dead: 49
their: 48
wife: 47
doth: 47
yet: 47
day: 46
can: 46
let: 45
tell: 45
time: 44
fair: 44
them: 44
upon: 44
these: 43
take: 42
like: 41
must: 41
tis: 41
di

# Question 3: Dataset: populationbycountry19802010millions.csv
Solve: For each year and region, compute percentage increase in population, year over year. Note the year 1980 will not have a preceding year.
For each year, display the top and bottom country in terms of global growth
Example:
Year, Region, yearly increase, percent of global year increase (these results are made up)
1981, North America, 1.30% 1981, Bermuda, 0.1% 1982, Aruba

In [ ]:
#get name of first column
first_col_name = data.columns[0]


data = data.withColumnRenamed(first_col_name, "Region")
data = data.replace('--', None)
data = data.dropna()

pct_increase = {}
for year in range(1981, 2011):
    year_str = str(year)
    prev_year_str = str(year - 1)
    data = data.withColumn(f"{year_str}_pct_increase", ((col(year_str) - col(prev_year_str)) / col(prev_year_str)))
    pct_increase[year_str] = f"{year_str}_pct_increase"

#compute % increase in population year over year for each region
for year in range(1981, 2011):
    year_str = str(year)
    yearly_increase = data.select(sum(pct_increase[year_str])).collect()[0][0]
    data = data.withColumn(f"{year_str}_global_increase", col(pct_increase[year_str]))

    filtered_data = data.filter(col(pct_increase[year_str]).isNotNull())

    top_row = filtered_data.orderBy(col(pct_increase[year_str]).desc()).select("Region", f"{year_str}_global_increase").first()
    top_country = top_row["Region"]
    top_global_pct = top_row[f"{year_str}_global_increase"]

    bottom_row = filtered_data.orderBy(col(pct_increase[year_str])).select("Region", f"{year_str}_global_increase").first()
    bottom_country = bottom_row["Region"]
    bottom_global_pct = bottom_row[f"{year_str}_global_increase"]

    print(f"{year}, {top_country}, {top_global_pct:.2%}, {bottom_country}, {bottom_global_pct:.2%}")


# Problem 5: Datasets: durham-nc-foreclosure-2006-2016.json Restaurants_in_Durham_County_NC.json
Solve: For each restaurant (‘Restaurants_in_Durham_County_NC.json’) classified as “status”:“ACTIVE” and “"rpt_area_desc": "Food Service":
For each restaurant, show the number of foreclosures (‘durham-nc-foreclosure-2006-2016’) within a radius of 10 miles of the restaurant’s coordinates.
Note: Assume the shape of Earth is a sphere. You can use the Haversine distance. https://pypi.org/project/haversine/

In [ ]:
from pyspark import SparkContext,SparkConf
fc = SparkConf()
fc.set("spark.submit.deployMode", "client")
cs = SparkContext.getOrCreate(fc)
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("temp").config("spark.some.config.option","some-value").getOrCreate()
restaurants = spark.read\
    .option("delimiter", ";")\
    .option("inferSchema", True)\
    .option("header", True)\
    .csv("Restaurants_in_Durham_County_NC.csv")\
    .repartition(10)

from pyspark.sql.functions import col
active = restaurants\
    .where((col("Status")=='ACTIVE') & (col("Rpt_Area_Desc")=='Food Service'))\
    .select("Premise_Name", "geolocation")

foreclosures = spark.read\
    .json("durham-nc-foreclosure-2006-2016.json")\
    .repartition(10)
result = active.select("Premise_Name", "geolocation")\
    .join(foreclosures.select("recordid", "geometry"), how="cross")
def string2array(x):
    if x is None:
        return None
    y = x.split(",")
    return [float(y[0]), float(y[1])]

def array2array(x):
    if x is None:
        return None
    return [x[1], x[0]]
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType

udf_string2array = udf(string2array, ArrayType(DoubleType()))
udf_fliparray = udf(array2array, ArrayType(DoubleType()))

result = result\
    .withColumn("geopoint1", udf_string2array(col("geolocation")))\
    .withColumn("geopoint2", udf_fliparray(col("geometry.coordinates")))\
    .select("Premise_Name", "geopoint1", "geopoint2")
from haversine import haversine

def haversine_miles(x, y):
    return haversine(x, y, unit='mi')

udf_haversine = udf(haversine_miles, DoubleType())
answer = result.dropna()\
    .select("Premise_Name", udf_haversine(col("geopoint1"), col("geopoint2")).alias("distance"))\
    .where(col("distance") <= 10.0)
answer.show(5)
answer.groupBy("Premise_Name").count()